In [1]:
# Cargando las librerias a usar

import pandas as pd
import numpy as np
import re, os
from string import printable
from sklearn import model_selection

#import gensim
import tensorflow as tf
from keras.models import Sequential, Model, model_from_json, load_model
from keras import regularizers
from keras.layers.core import Dense, Dropout, Activation, Lambda, Flatten
from keras.layers import Input, ELU, LSTM, Embedding, Convolution2D, MaxPooling2D, \
BatchNormalization, Convolution1D, MaxPooling1D, concatenate
from keras.preprocessing import sequence
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from keras import backend as K

from pathlib import Path
import json

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [130]:
## Cargar los datos de la base de datos
DATA_HOME = 'data/'
df = pd.read_csv(DATA_HOME + 'data_url.csv')

DATA_HOME = 'model/'
df.sample().head() 

,url,isMalicious
32656,translink.com.au/plan-your-journey/timetables/...,0


## Preparando la data (1 item)
Cada **caracter imprimible** tiene una representación en Ascii, pero para facilitar su uso en python , en vez de mostrarnos su valor en ASCII, lo que haremos será, mostrarnos el valor que ocupa en un diccionario ya definido en python

In [131]:
printable.index("g")

16

In [132]:
# Tengamos en cuenta que sumaremos 1 para que los valores no empiecen con 0

hola="google.com/N95"

url_int_tokens = [[(printable.index(x) + 1) for x in url if x in printable] for url in hola]
url_int_tokens

[[17],
 [25],
 [25],
 [17],
 [22],
 [15],
 [76],
 [13],
 [25],
 [23],
 [77],
 [50],
 [10],
 [6]]

## Preparando la data (Todos los items)
* Cada **caracter imprimible** tiene una representación en Ascii, pero para facilitar su uso en python , en vez de mostrarnos su valor en ASCII, lo que haremos será, mostrarnos el valor que ocupa en un diccionario ya definido en python
* Ademas vamos a restringir la cantidad de caracteres a 75, si son menos, se rellenaran con 0
* Luego pondremos todos esos valores en una matriz, es con esa matriz, con la cual vamos a trabajar

In [156]:
# Paso 1: Convierta una cadena URL sin procesar en la lista de listas donde los caracteres contenidos
#         en "imprimible" se almacenan codificados como enteros [ASCII]

url_int_tokens = [[(printable.index(x) + 1) for x in url if x in printable] for url in df.url]

In [157]:
# Paso 2: corte la cadena de URL en max_len o rellene con ceros si es más corter
"""Esta función transforma una lista (de longitud num_samples) de secuencias (listas de enteros) 
en una matriz 2D de Numpy de forma (num_samples, num_timesteps). num_timesteps es el argumento 
maxlen si se proporciona o la longitud de la secuencia más larga de la lista. Las secuencias que son más 
cortas que num_timesteps se rellenan con valor hasta que son num_timesteps de largo. Las secuencias más largas que
num_timesteps se truncan para que se ajusten a la longitud deseada. La posición donde ocurre el relleno o 
el truncamiento está determinada por los argumentos de relleno y trunca"""
max_len=100
X = sequence.pad_sequences(url_int_tokens, maxlen=max_len)

In [158]:
# Paso 3: Volcar las etiquetas de df a una matriz numpy

target = np.array(df.isMalicious)

print('Matrix dimensions of X: ', X.shape, 'Vector dimension of target: ', target.shape)

Matrix dimensions of X:  (212835, 100) Vector dimension of target:  (212835,)


In [159]:
# Simple Cross-Validation: Split the data set into training and test data
X_train, X_test, target_train, target_test = model_selection.train_test_split(X, target, test_size=0.25, random_state=33)

In [160]:
X_train[:5]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0, 14, 15, 12, 15, 29, 31, 13, 18, 15, 28, 77,
        32, 15, 28, 11, 24, 29, 30, 11, 22, 30, 31, 24, 17, 29, 14, 15,
        30, 11, 19, 22, 29, 77, 30, 15, 28, 23, 19, 24, 77,  2,  4, 10,
         2, 77, 11, 77, 29, 18, 25, 33, 15, 32, 15, 24, 30, 77, 13, 77,
        42, 11, 19, 28],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0, 23, 35, 30, 18, 19, 28, 30, 35, 25, 24, 15, 76, 13, 25,
        23, 77, 19, 24, 16, 25, 77, 44, 25, 29, 30, 15, 29, 29, 54, 15,
        33, 11, 28, 14, 29, 65, 18, 25, 29, 30, 15, 29, 29, 75, 13, 28,
        15, 14, 19, 30],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 15, 22, 26, 15, 28,
        19, 25

## Disposiciones generales para todos los modelos

In [161]:
# GENERAL get layer dimensions for any model!
def print_layers_dims(model):
    l_layers = model.layers
    # Note None is ALWAYS batch_size
    for i in range(len(l_layers)):
        print(l_layers[i])
        print('Input Shape: ', l_layers[i].input_shape, 'Output Shape: ', l_layers[i].output_shape)

# GENERAL save model to disk function!
def save_model(fileModelJSON,fileWeights):
    #print("Saving model to disk: ",fileModelJSON,"and",fileWeights)
    #have h5py installed
    if Path(fileModelJSON).is_file():
        os.remove(fileModelJSON)
    json_string = model.to_json()
    with open(fileModelJSON,'w' ) as f:
        json.dump(json_string, f)
    if Path(fileWeights).is_file():
        os.remove(fileWeights)
    model.save_weights(fileWeights)
    

# GENERAL load model from disk function!
def load_model(fileModelJSON,fileWeights):
    #print("Saving model to disk: ",fileModelJSON,"and",fileWeights)
    with open(fileModelJSON, 'r') as f:
         model_json = json.load(f)
         model = model_from_json(model_json)
    
    model.load_weights(fileWeights)
    return model

# Modelo 01- 1D Convolutions and Fully Connected Layers

## Entrenamiento

In [256]:
def conv_fully(max_len=100, emb_dim=50, max_vocab_len=100, W_reg=regularizers.l2(1e-4)):
    # Input
    main_input = Input(shape=(max_len,), dtype='int32', name='main_input')
    # Embedding layer
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len,
                W_regularizer=W_reg)(main_input) 
    emb = Dropout(0.25)(emb)

    
    def sum_1d(X):
        return K.sum(X, axis=1)
    
    def get_conv_layer(emb, kernel_size=5, filters=256):
        # Conv layer
        conv = Convolution1D(kernel_size=kernel_size, filters=filters, \
                     border_mode='same')(emb)
        conv = ELU()(conv)

        conv = Lambda(sum_1d, output_shape=(filters,))(conv)
        #conv = BatchNormalization(mode=0)(conv)
        conv = Dropout(0.5)(conv)
        return conv
        
    # Multiple Conv Layers
    
    # calling custom conv function from above
    conv1 = get_conv_layer(emb, kernel_size=2, filters=256)
    conv2 = get_conv_layer(emb, kernel_size=3, filters=256)
    conv3 = get_conv_layer(emb, kernel_size=4, filters=256)
    conv4 = get_conv_layer(emb, kernel_size=5, filters=256)

    # Fully Connected Layers (linear)
    merged = concatenate([conv1,conv2,conv3,conv4], axis=1)

    hidden1 = Dense(1024)(merged)
    hidden1 = ELU()(hidden1)
    hidden1 = BatchNormalization(mode=0)(hidden1)
    hidden1 = Dropout(0.5)(hidden1)

    hidden2 = Dense(1024)(hidden1)
    hidden2 = ELU()(hidden2)
    hidden2 = BatchNormalization(mode=0)(hidden2)
    hidden2 = Dropout(0.5)(hidden2)
    
    # Output layer (last fully connected layer)
    output = Dense(1, activation='sigmoid', name='output')(hidden2)

    # Compile model and define optimizer
    model = Model(input=[main_input], output=[output])
    adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [257]:
epochs = 20
batch_size = 32

model = conv_fully()
model.fit(X_train, target_train, epochs=epochs, batch_size=batch_size)
loss, accuracy = model.evaluate(X_test, target_test, verbose=1)

print('\nFinal Cross-Validation Accuracy', accuracy, '\n')
print_layers_dims(model)

Epoch 1/20
159626/159626 [==============================] - 1084s 7ms/step - loss: 0.6390 - accuracy: 0.6843
Epoch 2/20
159626/159626 [==============================] - 1078s 7ms/step - loss: 0.4955 - accuracy: 0.7590
Epoch 3/20
159626/159626 [==============================] - 1082s 7ms/step - loss: 0.4559 - accuracy: 0.7851
Epoch 4/20
159626/159626 [==============================] - 1045s 7ms/step - loss: 0.4358 - accuracy: 0.7963
Epoch 5/20
159626/159626 [==============================] - 1062s 7ms/step - loss: 0.4220 - accuracy: 0.8026
Epoch 6/20
159626/159626 [==============================] - 1189s 7ms/step - loss: 0.4138 - accuracy: 0.8067
Epoch 7/20
159626/159626 [==============================] - 1090s 7ms/step - loss: 0.4089 - accuracy: 0.8089
Epoch 8/20
159626/159626 [==============================] - 1112s 7ms/step - loss: 0.4014 - accuracy: 0.8140
Epoch 9/20
159626/159626 [==============================] - 1036s 6ms/step - loss: 0.3962 - accuracy: 0.8161
Epoch 10/20
159626/

### Comparativa con un modelo similar

``` conv_fully(max_len=75, emb_dim=32, max_vocab_len=100, W_reg=regularizers.l2(1e-4)):```


*Embedding layer
*Input_dim :: 100
*Output_dim :: 32
*Input_length :: 75
*epochs :: 20
*batch_size :: 32


### Final Cross-Validation Accuracy 0.8544043302536011 

```
<keras.engine.input_layer.InputLayer object at 0x00000128F4636D48>
Input Shape:  (None, 75) Output Shape:  (None, 75)
<keras.layers.embeddings.Embedding object at 0x00000128D2E9E388>
Input Shape:  (None, 75) Output Shape:  (None, 75, 32)
<keras.layers.core.Dropout object at 0x00000128D2E9E408>
Input Shape:  (None, 75, 32) Output Shape:  (None, 75, 32)
<keras.layers.convolutional.Conv1D object at 0x00000128D2E9EA48>
Input Shape:  (None, 75, 32) Output Shape:  (None, 75, 256)
<keras.layers.convolutional.Conv1D object at 0x00000128D2ED1588>
Input Shape:  (None, 75, 32) Output Shape:  (None, 75, 256)
<keras.layers.convolutional.Conv1D object at 0x00000128D2F094C8>
Input Shape:  (None, 75, 32) Output Shape:  (None, 75, 256)
<keras.layers.convolutional.Conv1D object at 0x00000128D2F41FC8>
Input Shape:  (None, 75, 32) Output Shape:  (None, 75, 256)
<keras.layers.advanced_activations.ELU object at 0x00000128D2E9E3C8>
Input Shape:  (None, 75, 256) Output Shape:  (None, 75, 256)
<keras.layers.advanced_activations.ELU object at 0x00000128D2EDB388>
Input Shape:  (None, 75, 256) Output Shape:  (None, 75, 256)
<keras.layers.advanced_activations.ELU object at 0x00000128D2F0B6C8>
Input Shape:  (None, 75, 256) Output Shape:  (None, 75, 256)
<keras.layers.advanced_activations.ELU object at 0x00000128D2F4F388>
Input Shape:  (None, 75, 256) Output Shape:  (None, 75, 256)
<keras.layers.core.Lambda object at 0x00000128D2EA7AC8>
Input Shape:  (None, 75, 256) Output Shape:  (None, 256)
<keras.layers.core.Lambda object at 0x00000128D2EDFB08>
Input Shape:  (None, 75, 256) Output Shape:  (None, 256)
<keras.layers.core.Lambda object at 0x00000128D2F17D08>
Input Shape:  (None, 75, 256) Output Shape:  (None, 256)
<keras.layers.core.Lambda object at 0x00000128D2F4EF08>
Input Shape:  (None, 75, 256) Output Shape:  (None, 256)
<keras.layers.core.Dropout object at 0x00000128D2EB4EC8>
Input Shape:  (None, 256) Output Shape:  (None, 256)
<keras.layers.core.Dropout object at 0x00000128D2EDF5C8>
Input Shape:  (None, 256) Output Shape:  (None, 256)
<keras.layers.core.Dropout object at 0x00000128D2F26088>
Input Shape:  (None, 256) Output Shape:  (None, 256)
<keras.layers.core.Dropout object at 0x00000128D2F5D408>
Input Shape:  (None, 256) Output Shape:  (None, 256)
<keras.layers.merge.Concatenate object at 0x00000128D2F82608>
Input Shape:  [(None, 256), (None, 256), (None, 256), (None, 256)] Output Shape:  (None, 1024)
<keras.layers.core.Dense object at 0x00000128D2F7D388>
Input Shape:  (None, 1024) Output Shape:  (None, 1024)
<keras.layers.advanced_activations.ELU object at 0x00000128D2FB1CC8>
Input Shape:  (None, 1024) Output Shape:  (None, 1024)
<keras.layers.normalization.BatchNormalization object at 0x00000128D2FB1D88>
Input Shape:  (None, 1024) Output Shape:  (None, 1024)
<keras.layers.core.Dropout object at 0x00000128D2FB1948>
Input Shape:  (None, 1024) Output Shape:  (None, 1024)
<keras.layers.core.Dense object at 0x00000128D2FBD188>
Input Shape:  (None, 1024) Output Shape:  (None, 1024)
<keras.layers.advanced_activations.ELU object at 0x00000128D2FDA748>
Input Shape:  (None, 1024) Output Shape:  (None, 1024)
<keras.layers.normalization.BatchNormalization object at 0x00000128D3042C08>
Input Shape:  (None, 1024) Output Shape:  (None, 1024)
<keras.layers.core.Dropout object at 0x00000128D3042948>
Input Shape:  (None, 1024) Output Shape:  (None, 1024)
<keras.layers.core.Dense object at 0x00000128D3070848>
Input Shape:  (None, 1024) Output Shape:  (None, 1)
```

In [258]:
model_name = "deeplearning_1DConv"
save_model(DATA_HOME + model_name + ".json", DATA_HOME + model_name + ".h5")
model = load_model(DATA_HOME + model_name + ".json", DATA_HOME + model_name + ".h5")
#print_layers_dims(model)

## Realizando una predicción

In [259]:
test_url_mal1 = "www.facel00k.com/mlost1/"
test_url_ben1 = "www.facebook.com/mlost1/"
test_url_mal2 = "naureen.net/etisalat.ae/index2.php"
test_url_ben2 = "sixt.com/php/reservation?language=en_US"

url = np.array([test_url_ben1,test_url_mal1,test_url_ben2,test_url_mal2])

#Tokenizamos
max_len=100
url_int_tokens = [[printable.index(x) + 1 for x in url if x in printable]for url in url]
X = sequence.pad_sequences(url_int_tokens, maxlen=max_len)

In [260]:
proba = model.predict(X, batch_size=1)
def print_result(proba,x):
    if float(proba[x]) > 0.5:
        return "no es segura"
    else:
        return "es segura"


    
for x in range(len(url)):
    print("La página",url[x],"\t", print_result(proba,x),"\n\t con la probabilidad de que sea phishing del ",\
          round(float(proba[x]),2), "%" )

La página www.facebook.com/mlost1/ 	 no es segura 
	 con la probabilidad de que sea phishing del  0.74 %
La página www.facel00k.com/mlost1/ 	 no es segura 
	 con la probabilidad de que sea phishing del  0.67 %
La página sixt.com/php/reservation?language=en_US 	 es segura 
	 con la probabilidad de que sea phishing del  0.08 %
La página naureen.net/etisalat.ae/index2.php 	 no es segura 
	 con la probabilidad de que sea phishing del  0.88 %


In [261]:
# get for example word2vec embedding weight matix
# l_layers = model.layers
# weights = l_layers[1].get_weights()
# weights[0].shape

# Modelo 02 - 1D Convolution and LSTM


## Entrenamiento

In [162]:
# Embedding layer
#  Input_dim :: Cantidad de Caracteres difirentes a utilizar palabras en el diccionario
#  Output_dim :: Dimensión de un vector de incrustación (para cada caracter seleccionado)
#  Input_length :: Cantidad de caracteres que se van a recibir (Es decir cantidad de caracteres que se leeran para la predicción)

def lstm_conv(max_len=100, emb_dim=50, max_vocab_len=100, lstm_output_size=50, W_reg=regularizers.l2(1e-4)):
    
    # Input
    main_input = Input(shape=(max_len,), dtype='int32', name='main_input')
    
    # Embedding layer
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len,
                W_regularizer=W_reg)(main_input) 
    emb = Dropout(0.25)(emb)

    # Conv layer
    conv = Convolution1D(kernel_size=5, filters=256, \
                     border_mode='same')(emb)
    conv = ELU()(conv)

    conv = MaxPooling1D(pool_size=4)(conv)
    conv = Dropout(0.5)(conv)

    # LSTM layer
    lstm = LSTM(lstm_output_size)(conv)
    lstm = Dropout(0.5)(lstm)
    
    # Output layer (last fully connected layer)
    output = Dense(1, activation='sigmoid', name='output')(lstm)

    # Compile model and define optimizer
    model = Model(input=[main_input], output=[output])
    adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [163]:
epochs = 20
batch_size = 32

model2 = lstm_conv()
model2.fit(X_train, target_train, epochs=epochs, batch_size=batch_size)
loss, accuracy = model2.evaluate(X_test, target_test, verbose=1)



Epoch 1/20
159626/159626 [==============================] - 586s 4ms/step - loss: 0.4099 - accuracy: 0.8110
Epoch 2/20
159626/159626 [==============================] - 658s 4ms/step - loss: 0.3020 - accuracy: 0.8745
Epoch 3/20
159626/159626 [==============================] - 609s 4ms/step - loss: 0.2670 - accuracy: 0.8926
Epoch 4/20
159626/159626 [==============================] - 566s 4ms/step - loss: 0.2437 - accuracy: 0.9018
Epoch 5/20
159626/159626 [==============================] - 596s 4ms/step - loss: 0.2294 - accuracy: 0.9084
Epoch 6/20
159626/159626 [==============================] - 616s 4ms/step - loss: 0.2173 - accuracy: 0.9142
Epoch 7/20
159626/159626 [==============================] - 645s 4ms/step - loss: 0.2098 - accuracy: 0.9163
Epoch 8/20
159626/159626 [==============================] - 636s 4ms/step - loss: 0.2023 - accuracy: 0.9204
Epoch 9/20
159626/159626 [==============================] - 586s 4ms/step - loss: 0.1972 - accuracy: 0.9225
Epoch 10/20
159626/159626 [=

In [164]:
print('\nFinal Cross-Validation Accuracy', accuracy, '\n')
print_layers_dims(model2)


Final Cross-Validation Accuracy 0.9496889710426331 

Input Shape:  (None, 100) Output Shape:  (None, 100)
Input Shape:  (None, 100) Output Shape:  (None, 100, 50)
Input Shape:  (None, 100, 50) Output Shape:  (None, 100, 50)
Input Shape:  (None, 100, 50) Output Shape:  (None, 100, 256)
Input Shape:  (None, 100, 256) Output Shape:  (None, 100, 256)
Input Shape:  (None, 100, 256) Output Shape:  (None, 25, 256)
Input Shape:  (None, 25, 256) Output Shape:  (None, 25, 256)
Input Shape:  (None, 25, 256) Output Shape:  (None, 50)
Input Shape:  (None, 50) Output Shape:  (None, 50)
Input Shape:  (None, 50) Output Shape:  (None, 1)


### Comparativa con un modelo similar

```lstm_conv(max_len=75, emb_dim=32, max_vocab_len=100, lstm_output_size=32, W_reg=regularizers.l2(1e-4))```


*Embedding layer
*Input_dim :: 100
*Output_dim :: 32
*Input_length :: 75
*epochs :: 20
*batch_size :: 32


### Final Cross-Validation Accuracy 0.9385442137718201 


```<keras.engine.input_layer.InputLayer object at 0x00000128D598FC88>
Input Shape:  (None, 75) Output Shape:  (None, 75)
<keras.layers.embeddings.Embedding object at 0x00000128D59957C8>
Input Shape:  (None, 75) Output Shape:  (None, 75, 32)
<keras.layers.core.Dropout object at 0x00000128D598F808>
Input Shape:  (None, 75, 32) Output Shape:  (None, 75, 32)
<keras.layers.convolutional.Conv1D object at 0x00000128D598F388>
Input Shape:  (None, 75, 32) Output Shape:  (None, 75, 256)
<keras.layers.advanced_activations.ELU object at 0x00000128D598F8C8>
Input Shape:  (None, 75, 256) Output Shape:  (None, 75, 256)
<keras.layers.pooling.MaxPooling1D object at 0x00000128D5991B88>
Input Shape:  (None, 75, 256) Output Shape:  (None, 18, 256)
<keras.layers.core.Dropout object at 0x00000128D6642808>
Input Shape:  (None, 18, 256) Output Shape:  (None, 18, 256)
<keras.layers.recurrent.LSTM object at 0x00000128D664AE88>
Input Shape:  (None, 18, 256) Output Shape:  (None, 32)
<keras.layers.core.Dropout object at 0x00000128D664DD48>
Input Shape:  (None, 32) Output Shape:  (None, 32)
<keras.layers.core.Dense object at 0x00000128D6654C88>
Input Shape:  (None, 32) Output Shape:  (None, 1)
``` 

In [174]:
model_name = "deeplearning_1DConvLSTM"
save_model(DATA_HOME + model_name + ".json", DATA_HOME + model_name + ".h5")
model = load_model(DATA_HOME + model_name + ".json", DATA_HOME + model_name + ".h5")
#print_layers_dims(model)

## Realizando una predicción


In [271]:
test_url_mal1 = "www.facel00k.com/mlost1/"
test_url_ben1 = "www.facebook.com/mlost1/"
test_url_mal2 = "naureen.net/etisalat.ae/index2.php"
test_url_ben2 = "sixt.com/php/reservation?language=en_US"
test_url_mal3 = "tottus.supercupones.net"
test_url_mal4 = "netfIix.com"

url = np.array([test_url_ben1,test_url_mal1,test_url_ben2,test_url_mal2,test_url_mal3,test_url_mal4])

#Tokenizamos
max_len=100
url_int_tokens = [[printable.index(x) + 1 for x in url if x in printable]for url in url]
X = sequence.pad_sequences(url_int_tokens, maxlen=max_len)


In [272]:
proba = model2.predict(X, batch_size=1)
def print_result(proba,x):
    if float(proba[x]) > 0.5:
        return "no es segura"
    else:
        return "es segura"


    
for x in range(len(url)):
    print("La página",url[x],"\t", print_result(proba,x),"\n\t con la probabilidad de que sea phishing del ",\
          round(float(proba[x]),2), "%" )

La página www.facebook.com/mlost1/ 	 es segura 
	 con la probabilidad de que sea phishing del  0.34 %
La página www.facel00k.com/mlost1/ 	 no es segura 
	 con la probabilidad de que sea phishing del  0.88 %
La página sixt.com/php/reservation?language=en_US 	 es segura 
	 con la probabilidad de que sea phishing del  0.0 %
La página naureen.net/etisalat.ae/index2.php 	 no es segura 
	 con la probabilidad de que sea phishing del  0.99 %
La página netflix.com 	 no es segura 
	 con la probabilidad de que sea phishing del  0.69 %
La página tottus.supercupones.net 	 no es segura 
	 con la probabilidad de que sea phishing del  0.92 %
La página netfIix.com 	 no es segura 
	 con la probabilidad de que sea phishing del  0.73 %
